# nb-model

In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
from functools import partial, reduce
import logging

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname

def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fname = 'nb-model_xg0.ipynb'   # FILL
dir_name = 'model'             # FILL
fix_path(get_cwd(fname, dir_name +sep))

import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from dask import delayed, compute
from torch.utils.data import TensorDataset, DataLoader
import torch

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', 50)

from common_util import RECON_DIR, JSON_SFX_LEN, DT_CAL_DAILY_FREQ, is_type, pd_common_idx_rows, remove_dups_list, set_loglevel, search_df, chained_filter, get_variants, dump_df, load_json, gb_transpose, pd_common_index_rows, filter_cols_below, inner_join, outer_join, ser_shift, list_get_dict, window_iter, benchmark
from common_util import midx_get_level, pd_rows, midx_intersect, pd_common_idx_rows, midx_split, pd_midx_to_arr, window_iter, np_is_ndim, identity_fn
from model.common import DATASET_DIR, XG_PROCESS_DIR, XG_DATA_DIR, XG_DIR, EXPECTED_NUM_HOURS, default_dataset
from model.dataprep_util import align_first_last_cols, prune_nulls
from model.datagen_util import xgdg, process_group
from data.data_api import DataAPI
from recon.dataset_util import prep_dataset, gen_group
from model.dataprep_util import COMMON_PREP_MAPPING, DATA_PREP_MAPPING
#from recon.split_util import get_train_test_split, gen_time_series_split, index_three_split, pd_binary_clip
DataAPI.__init__()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

CRITICAL:root:script location: /home/kev/crunch/model/nb-model_xg0.ipynb
CRITICAL:root:using project dir: /home/kev/crunch/


## Load

In [9]:
xg_fname = 'dohlca.json'
xg_kc = ['sp_500', 'dohlca', 'dohlca_pba_vol', 'pba_dohlca', 'pba_dohlca']

In [14]:
xg_fname = 'ddiff.json'
xg_kc = ['sp_500', 'ddiff', 'ddiff_pba_vol', 'pba_dohlca_ddiff', 'pba_dohlca_ddiff(1)']

In [15]:
for parent_path, dirs, files in os.walk(XG_PROCESS_DIR, topdown=False):
    if (xg_fname in files):
        xg_dict = load_json(xg_fname, dir_path=parent_path)
        dataset_dict = load_json(xg_dict['dataset'], dir_path=parent_path) if (is_type(xg_dict['dataset'], str)) else xg_dict['dataset']
        dataset = prep_dataset(dataset_dict, assets=None)
        ex = dataset['features']['dfs'][xg_kc].compute()
        break

/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


## Test FN

In [16]:
from common_util import pd_dti_idx_date_only, pd_idx_rename, is_ser, dcompose, compose, pd_idx_to_midx

In [17]:
def prep_stack_data(feature_df, delayed=False):
	"""
	Converts a single indexed daily DataFrame into a MultiIndexed daily DataFrame.

	Args:
		feature_df (pd.DataFrame): Daily DataFrame
		row_masks_df (pd.DataFrame): DataFrame of row masks / time mask
		delayed (boolean): Whether or not to create a delayed function composition

	Returns:
		pd.DataFrame or dask Delayed object
	"""
	preproc = (
				pd_dti_idx_date_only,		# Removes the time component of the DatetimeIndex index
				partial(pd_idx_to_midx, col_name=-1)
			)
	prep_fn = dcompose(*preproc) if (delayed) else compose(*preproc)
	return prep_fn(feature_df)

In [18]:
prep_stack_data(ex)

-1
id0        id1                    
2009-01-05 pba_open      26.180000
           pba_high       1.900000
           pba_low       20.180000
           pba_close     -4.350000
           pba_avgPrice   8.173325
...                            ...
2017-12-29 pba_open       3.050000
           pba_high       4.460000
           pba_low       -9.080000
           pba_close    -13.930000
           pba_avgPrice  -1.129550

[11320 rows x 1 columns]